# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 9:47AM,257917,18,EMERSA-9191745,"Emersa Waterbox, LLC",Released
1,Feb 22 2023 9:42AM,257916,22,642058,"NBTY Global, Inc.",Released
2,Feb 22 2023 9:33AM,257915,10,LUPLUP0018,"Lupin Research, Inc.",Released
3,Feb 22 2023 9:30AM,257890,15,30014112,"Alliance Pharma, Inc.",Released
4,Feb 22 2023 9:30AM,257890,15,30014113,"Alliance Pharma, Inc.",Released
5,Feb 22 2023 9:30AM,257890,15,30014114,"Alliance Pharma, Inc.",Released
6,Feb 22 2023 9:30AM,257890,15,30014115,"Alliance Pharma, Inc.",Released
7,Feb 22 2023 9:30AM,257890,15,30014116,"Alliance Pharma, Inc.",Released
8,Feb 22 2023 9:30AM,257890,15,30014117,"Alliance Pharma, Inc.",Released
9,Feb 22 2023 9:30AM,257890,15,30014178,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,257913,Released,1
43,257914,Released,20
44,257915,Released,1
45,257916,Released,1
46,257917,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257913,NaN,NaN,1.0
257914,NaN,NaN,20.0
257915,NaN,NaN,1.0
257916,NaN,NaN,1.0
257917,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257817,0.0,0.0,8.0
257825,0.0,0.0,1.0
257832,4.0,0.0,4.0
257837,1.0,0.0,2.0
257839,0.0,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257817,0,0,8
257825,0,0,1
257832,4,0,4
257837,1,0,2
257839,0,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257817,0,0,8
1,257825,0,0,1
2,257832,4,0,4
3,257837,1,0,2
4,257839,0,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257817,,,8
1,257825,,,1
2,257832,4,,4
3,257837,1,,2
4,257839,,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC"
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc."
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc."
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc."
72,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation
92,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc."
93,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated"
94,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC"
95,Feb 22 2023 9:10AM,257910,10,Methapharm-G
96,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC",,,1
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc.",,,1
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc.",,,1
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc.",,,69
4,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation,,,20
5,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",,,1
6,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",,,1
7,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",,,1
8,Feb 22 2023 9:10AM,257910,10,Methapharm-G,,,2
9,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC",1,,
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc.",1,,
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc.",1,,
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc.",69,,
4,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation,20,,
5,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",1,,
6,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",1,,
7,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",1,,
8,Feb 22 2023 9:10AM,257910,10,Methapharm-G,2,,
9,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC",1,,
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc.",1,,
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc.",1,,
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc.",69,,
4,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc.",1.0,NaN,NaN
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc.",69.0,NaN,NaN
4,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:47AM,257917,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Feb 22 2023 9:42AM,257916,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Feb 22 2023 9:33AM,257915,10,"Lupin Research, Inc.",1.0,0.0,0.0
3,Feb 22 2023 9:30AM,257890,15,"Alliance Pharma, Inc.",69.0,0.0,0.0
4,Feb 22 2023 9:26AM,257914,10,ISDIN Corporation,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7220604,158.0,10.0,18.0
12,1031604,7.0,2.0,0.0
15,257890,69.0,0.0,0.0
18,257917,1.0,0.0,0.0
19,1031511,24.0,11.0,0.0
21,515718,2.0,0.0,0.0
22,773663,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7220604,158.0,10.0,18.0
1,12,1031604,7.0,2.0,0.0
2,15,257890,69.0,0.0,0.0
3,18,257917,1.0,0.0,0.0
4,19,1031511,24.0,11.0,0.0
5,21,515718,2.0,0.0,0.0
6,22,773663,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,158.0,10.0,18.0
1,12,7.0,2.0,0.0
2,15,69.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,24.0,11.0,0.0
5,21,2.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,158.0
1,12,Released,7.0
2,15,Released,69.0
3,18,Released,1.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,21,22
Status,,,,,,,
Completed,18.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,2.0,0.0,0.0,11.0,0.0,0.0
Released,158.0,7.0,69.0,1.0,24.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,2.0,0.0,0.0,11.0,0.0,0.0
2,Released,158.0,7.0,69.0,1.0,24.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,2.0,0.0,0.0,11.0,0.0,0.0
2,Released,158.0,7.0,69.0,1.0,24.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()